In [6]:
import locator
import pyspark
from pyspark.sql.functions import udf,col, countDistinct
from pyspark.sql.types import StringType,FloatType,DoubleType

#Extra input dataset
file_location = "starbucks_store_locator.csv"
file_type = "csv"
df_starbucks_stores = locator.get_dataset(file_location, file_type, delimiter=",")

# Exploring the dataset
df_starbucks_stores.printSchema()
df_starbucks_stores.show(5)

#Clean dataset.Removed records with null longitudes/latitudes
df_starbucks_stores = df_starbucks_stores.filter((col('Longitude').isNotNull()) | (col('Latitude').isNotNull()))

#Convert lat/long values from string to float for easy computation later on
df_starbucks_stores = df_starbucks_stores\
                      .withColumn("Longitude", df_starbucks_stores["Longitude"].cast(FloatType()))\
                      .withColumn("Latitude", df_starbucks_stores["Latitude"].cast(FloatType()))

#Augment data with country information
df_country_info = locator.get_country_info()

#Join starbucks data + country info
df_starbucks_country_joined = df_starbucks_stores.join(df_country_info,\
        df_starbucks_stores.Country == df_country_info.ALPHA2,how='left')

df_with_null_countries = df_starbucks_country_joined.filter(col('ALPHA2').isNull())
df_starbucks_stores_with_country_name = df_starbucks_country_joined.filter(col('ALPHA2').isNotNull())


root
 |-- Brand: string (nullable = true)
 |-- Store Number: string (nullable = true)
 |-- Store Name: string (nullable = true)
 |-- Ownership Type: string (nullable = true)
 |-- Street Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State/Province: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Phone Number: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)

+---------+------------+----------------+--------------+--------------------+----------------+--------------+-------+--------+------------+--------------------+---------+--------+
|    Brand|Store Number|      Store Name|Ownership Type|      Street Address|            City|State/Province|Country|Postcode|Phone Number|            Timezone|Longitude|Latitude|
+---------+------------+----------------+--------------+--------------------+----------------

In [7]:
#1) Which is the most isolated Starbucks store in Australia ?

'''
Assumptions

1. Most Isolated Store/Branch - It means, here which store or branch  has the minimum distance between the next store/branch
   is the largest among its all neighbooring stores/branches.
2. Null valued longitude/latitude will be removed
3. Minimum distance is express in KM
4. Assume Earth is elliptical (Using Haversine distance formula)
'''
import locator
isolated = locator.get_most_isolated_store_per_country(df_starbucks_stores_with_country_name,'AU')
print('The most isolated Starbucks store in Australia is:\n')
print('Store Name: {}\nStore Number: {}\nNearest Branch: {}\nDistance(KM): {}\n'\
      .format(isolated['store_name'], isolated['store_number'], isolated['nearest_store'],isolated['nearest_distance_km']))


The most isolated Starbucks store in Australia is:

Store Name: Mount Druitt
Store Number: 25291-240407
Nearest Branch: Westmead Childrens Hospital
Distance(KM): 16.1



In [3]:
#2) Which country has the least number of Starbucks stores ?

#get_store_count_per_country(df_starbucks_stores_with_country_name)
#get_country_store_count(df_starbucks_stores_with_country_name,'AU')
import locator
print ('Countries with least number of Starbucks stores:')
least_stores = locator.get_countries_with_least_starbucks_stores(df_starbucks_stores_with_country_name)
for row in least_stores:
    print ('\nName: {}\nStore Counts: {}\n'.format(row.Country_Name,row.Store_Count))

Countries with least number of Starbucks stores:

Name: Andorra
Store Counts: 1


Name: Niger
Store Counts: 1



In [10]:
#3) How many stores in Sydney has not updated the "Phone Number" in this dataset ?
import locator
df_starbucks_au = locator.filter_data(df_starbucks_stores_with_country_name, 'Country', 'AU') 
df_starbucks_au.show()
print (df_starbucks_au.count())

+---------+------------+--------------------+--------------+--------------------+----------------+--------------+-------+--------+------------+--------------------+---------+--------+------------+------+------+
|    Brand|Store Number|          Store Name|Ownership Type|      Street Address|            City|State/Province|Country|Postcode|Phone Number|            Timezone|Longitude|Latitude|Country_Name|ALPHA2|ALPHA3|
+---------+------------+--------------------+--------------+--------------------+----------------+--------------+-------+--------+------------+--------------------+---------+--------+------------+------+------+
|Starbucks|49778-271255|               Manly|      Licensed|   110-112 The Corso|           Manly|           NSW|     AU|    2095|        null|GMT+10:00 Austral...|   151.29|   -33.8|   Australia|    AU|   AUS|
|Starbucks|25291-240407|        Mount Druitt|      Licensed|     Carlisle Avenue|       Mt Druitt|           NSW|     AU|    2770|        null|GMT+10:00 Aus